# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f46806607c0>
├── 'a' --> tensor([[-0.7249, -0.1447, -1.1540],
│                   [ 0.5689,  1.0123, -1.6154]])
└── 'x' --> <FastTreeValue 0x7f46806605b0>
    └── 'c' --> tensor([[-0.2795, -0.1338, -0.3619, -0.8739],
                        [-0.4648, -0.9925, -0.7018, -0.6755],
                        [ 0.5034,  0.3455,  0.3521,  0.2881]])

In [4]:
t.a

tensor([[-0.7249, -0.1447, -1.1540],
        [ 0.5689,  1.0123, -1.6154]])

In [5]:
%timeit t.a

75.6 ns ± 0.17 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f46806607c0>
├── 'a' --> tensor([[-0.8400,  0.2949, -1.7894],
│                   [ 0.7192, -0.1646,  0.7761]])
└── 'x' --> <FastTreeValue 0x7f46806605b0>
    └── 'c' --> tensor([[-0.2795, -0.1338, -0.3619, -0.8739],
                        [-0.4648, -0.9925, -0.7018, -0.6755],
                        [ 0.5034,  0.3455,  0.3521,  0.2881]])

In [7]:
%timeit t.a = new_value

91.2 ns ± 0.177 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.7249, -0.1447, -1.1540],
               [ 0.5689,  1.0123, -1.6154]]),
    x: Batch(
           c: tensor([[-0.2795, -0.1338, -0.3619, -0.8739],
                      [-0.4648, -0.9925, -0.7018, -0.6755],
                      [ 0.5034,  0.3455,  0.3521,  0.2881]]),
       ),
)

In [10]:
b.a

tensor([[-0.7249, -0.1447, -1.1540],
        [ 0.5689,  1.0123, -1.6154]])

In [11]:
%timeit b.a

69.8 ns ± 0.193 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.2062, -0.0082, -0.3642],
               [-1.3329,  0.5946, -0.6323]]),
    x: Batch(
           c: tensor([[-0.2795, -0.1338, -0.3619, -0.8739],
                      [-0.4648, -0.9925, -0.7018, -0.6755],
                      [ 0.5034,  0.3455,  0.3521,  0.2881]]),
       ),
)

In [13]:
%timeit b.a = new_value

594 ns ± 1.28 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

989 ns ± 2.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.9 µs ± 47.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

170 µs ± 743 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

164 µs ± 327 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f4680660c70>
├── 'a' --> tensor([[[-0.7249, -0.1447, -1.1540],
│                    [ 0.5689,  1.0123, -1.6154]],
│           
│                   [[-0.7249, -0.1447, -1.1540],
│                    [ 0.5689,  1.0123, -1.6154]],
│           
│                   [[-0.7249, -0.1447, -1.1540],
│                    [ 0.5689,  1.0123, -1.6154]],
│           
│                   [[-0.7249, -0.1447, -1.1540],
│                    [ 0.5689,  1.0123, -1.6154]],
│           
│                   [[-0.7249, -0.1447, -1.1540],
│                    [ 0.5689,  1.0123, -1.6154]],
│           
│                   [[-0.7249, -0.1447, -1.1540],
│                    [ 0.5689,  1.0123, -1.6154]],
│           
│                   [[-0.7249, -0.1447, -1.1540],
│                    [ 0.5689,  1.0123, -1.6154]],
│           
│                   [[-0.7249, -0.1447, -1.1540],
│                    [ 0.5689,  1.0123, -1.6154]]])
└── 'x' --> <FastTreeValue 0x7f45dbf2dfd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

38.3 µs ± 98.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4680653340>
├── 'a' --> tensor([[-0.7249, -0.1447, -1.1540],
│                   [ 0.5689,  1.0123, -1.6154],
│                   [-0.7249, -0.1447, -1.1540],
│                   [ 0.5689,  1.0123, -1.6154],
│                   [-0.7249, -0.1447, -1.1540],
│                   [ 0.5689,  1.0123, -1.6154],
│                   [-0.7249, -0.1447, -1.1540],
│                   [ 0.5689,  1.0123, -1.6154],
│                   [-0.7249, -0.1447, -1.1540],
│                   [ 0.5689,  1.0123, -1.6154],
│                   [-0.7249, -0.1447, -1.1540],
│                   [ 0.5689,  1.0123, -1.6154],
│                   [-0.7249, -0.1447, -1.1540],
│                   [ 0.5689,  1.0123, -1.6154],
│                   [-0.7249, -0.1447, -1.1540],
│                   [ 0.5689,  1.0123, -1.6154]])
└── 'x' --> <FastTreeValue 0x7f4680659b20>
    └── 'c' --> tensor([[-0.2795, -0.1338, -0.3619, -0.8739],
                        [-0.4648, -0.9925, -0.7018, -0.6755],
                 

In [23]:
%timeit t_cat(trees)

36.5 µs ± 168 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

79 µs ± 210 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.7249, -0.1447, -1.1540],
                [ 0.5689,  1.0123, -1.6154]],
       
               [[-0.7249, -0.1447, -1.1540],
                [ 0.5689,  1.0123, -1.6154]],
       
               [[-0.7249, -0.1447, -1.1540],
                [ 0.5689,  1.0123, -1.6154]],
       
               [[-0.7249, -0.1447, -1.1540],
                [ 0.5689,  1.0123, -1.6154]],
       
               [[-0.7249, -0.1447, -1.1540],
                [ 0.5689,  1.0123, -1.6154]],
       
               [[-0.7249, -0.1447, -1.1540],
                [ 0.5689,  1.0123, -1.6154]],
       
               [[-0.7249, -0.1447, -1.1540],
                [ 0.5689,  1.0123, -1.6154]],
       
               [[-0.7249, -0.1447, -1.1540],
                [ 0.5689,  1.0123, -1.6154]]]),
    x: Batch(
           c: tensor([[[-0.2795, -0.1338, -0.3619, -0.8739],
                       [-0.4648, -0.9925, -0.7018, -0.6755],
                       [ 0.5034,  0.3455,  0.3521,  0.2881]],
         

In [26]:
%timeit Batch.stack(batches)

90.4 µs ± 428 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.7249, -0.1447, -1.1540],
               [ 0.5689,  1.0123, -1.6154],
               [-0.7249, -0.1447, -1.1540],
               [ 0.5689,  1.0123, -1.6154],
               [-0.7249, -0.1447, -1.1540],
               [ 0.5689,  1.0123, -1.6154],
               [-0.7249, -0.1447, -1.1540],
               [ 0.5689,  1.0123, -1.6154],
               [-0.7249, -0.1447, -1.1540],
               [ 0.5689,  1.0123, -1.6154],
               [-0.7249, -0.1447, -1.1540],
               [ 0.5689,  1.0123, -1.6154],
               [-0.7249, -0.1447, -1.1540],
               [ 0.5689,  1.0123, -1.6154],
               [-0.7249, -0.1447, -1.1540],
               [ 0.5689,  1.0123, -1.6154]]),
    x: Batch(
           c: tensor([[-0.2795, -0.1338, -0.3619, -0.8739],
                      [-0.4648, -0.9925, -0.7018, -0.6755],
                      [ 0.5034,  0.3455,  0.3521,  0.2881],
                      [-0.2795, -0.1338, -0.3619, -0.8739],
                      [-0.4648, -

In [28]:
%timeit Batch.cat(batches)

168 µs ± 550 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

391 µs ± 14.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
